# M3.3 - Tracking Changes to Research Code

*Part of:* [**Open Science for Water Resources**](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources)

All we did was move the `import` statements to the top of the code block, where they belong.

In [2]:
import calendar
import datetime
import glob
import earthaccess
import numpy as np
import h5py
import xarray as xr
import geopandas
from matplotlib import pyplot
from pyproj import CRS

auth = earthaccess.login()

basin = geopandas.read_file('/home/arthur.endsley/Workspace/NTSG/projects/Y2024_TOPS_Training/data/YellowstoneRiver_drainage_WSG84.shp')

results = earthaccess.search_data(
    short_name = 'GPM_3IMERGM',
    temporal = ('2014-01-01', '2018-12-31'))

earthaccess.download(results, 'data_raw/IMERG-Final')

file_list = glob.glob('data_raw/IMERG-Final/*.HDF5')
file_list.sort()

datasets = []
for filename in file_list:
    date = datetime.datetime.strptime(filename.split('.')[4][0:8], '%Y%m%d')
    ds0 = xr.open_dataset(filename, group = 'Grid').get(['precipitation'])

    # Define the coordinate reference system (CRS) and the spatial coordinates
    ds0 = ds0.rio.write_crs(CRS.from_epsg(4326))
    ds0 = ds0.rio.set_spatial_dims('lon', 'lat')

    ds_clip = ds0.rio.clip(basin.geometry.values, basin.crs, from_disk = True)
    
    # Only write the file if it doesn't exist (in case we run this again)
    datasets.append(ds_clip)

ds = xr.concat(datasets, dim = 'time')

precip_series = ds.precipitation.sum(['lon','lat']).values
days_in_month = np.array(calendar.mdays)[ds.coords['time.month'].values]
precip_total = precip_series * days_in_month

Granules found: 60
 Getting 60 granules, approx download size: 1.04 GB


QUEUEING TASKS | :   0%|          | 0/60 [00:00<?, ?it/s]

File 3B-MO.MS.MRG.3IMERG.20180201-S000000-E235959.02.V07B.HDF5 already downloadedFile 3B-MO.MS.MRG.3IMERG.20180101-S000000-E235959.01.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20180301-S000000-E235959.03.V07B.HDF5 already downloaded

File 3B-MO.MS.MRG.3IMERG.20180701-S000000-E235959.07.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20180501-S000000-E235959.05.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20180401-S000000-E235959.04.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20180601-S000000-E235959.06.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20180901-S000000-E235959.09.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20181001-S000000-E235959.10.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20181101-S000000-E235959.11.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20181201-S000000-E235959.12.V07B.HDF5 already downloaded
File 3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5 already downloaded
File 3B-MO.MS.MR

PROCESSING TASKS | :   0%|          | 0/60 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/60 [00:00<?, ?it/s]